# Получение информации (индексов) с космоснимков

In [ ]:
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import ee

### Авторизация Google Earth Engine API

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
# coordinates region of interest (lat, lon)
df_generated_points = pd.read_csv('generated_points.csv')

### Апробация

Чтобы осуществить прогнозирование выбирается металл, по которому оно будет проводится, после чего, необходимо выбрать интересующую область. Важно отметить, концентрация металлов актуальна только для областей на земле, поэтому необходимо отсечь точки, лежащие на воде и заснеженных районах. Поскольку получение индексов — это время затратный процесс, перед получением целевых индексов, необходимо исключить неподходящие точки.
Для фильтрации точек используется нормализованный относительный индекс растительности. Простой показатель количества фотосинтетически активной биомассы (NDVI). Его показатели лежат в пределах . Области с густым растительным покровом будет стремиться к положительным значениям (скажем, от 0,3 до 0,8), а облака и снежные поля будут характеризоваться отрицательными значениями этого показателя. Свободно стоящая вода (например, океаны, моря, озера и реки) имеет довольно низкую отражательную способность в обоих спектральных диапазонах (по крайней мере, вдали от берегов), что приводит к очень низким положительным или даже отрицательным значениям NDVI.

### Алгоритм получения данных с космоснимков
Для получения индексов необходимо: выбрать точки пробоотбора, выбрать косми-ческую программу (коллекцию), определить область (в метрах), по которой будет произ-водиться анализ, выбрать временной промежуток, выбрать необходимый тип Reducer. 
Из полученных параметров формируется API-запрос на сервис GEE. После обра-ботки, сервис возвращает полученные индексы. В дальнейшем полученные индексы должны быть использованы для обучения моделей. Но перед тем, как обучать модели, нужно их отфильтровать.

В качестве примера продемонстрирован алгоритм получения NDVI.


### NDVI
Вегетационный индекс использует свойства красного каналов (кото-рый поглощает растительность) и ближних инфракрасных каналов (которые сильно отражают растительность). Как следует из названия, он используется для мониторинга здоровья и силы растительности. Нормализованный относительный индекс растительности (NDVI) всегда колеблется от -1 до +1. Отрицательные значения указывают на наличие воды и влаги. Но высокие значения NDVI предполагают густой растительный покров. Данный индекс активно примется в сельском хозяйстве для контроля состояния посевов.


![This is a alt text.](image/ndvi.jpg "This is a sample image.")

С использованием платформы GEE, были интерактивно получены показатели NDVI в различных районах, как можно увидеть на рисунке 61. Подобрано оптимально значение параметра, при котором отсутствуют точки на воде и в заснеженных районах. Точки отфильтрованы по порогу .

![This is a alt text.](image/gee_ndvi.png "This is a sample image.")

In [ ]:
def get_ndvi_index(df, image_collection, start_date, end_date):
    """
    Get the index of the generated points.
    """
    ndvi_val = []

    for i in tqdm(range(len(df))):
        lon = df['Longitude'][i]
        lat = df['Latitude'][i]
        point = (ee.Geometry.Point(lon, lat))

        l8 = ee.ImageCollection(image_collection)
    
        image = ee.Image(l8.filterBounds(point)
            .filterDate(start_date, end_date)
            .sort('CLOUD_COVER')
            .first())
    
        nir = image.select('B5')
        red = image.select('B4')
        ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
        val = (ndvi.reduceRegion(
                            reducer=ee.Reducer.mean(),
                            geometry=point,
                            scale=10))
        
        try:
            ndvi_val.append(val.getInfo().get('NDVI'))
        except:
            ndvi_val.append(np.nan)
            continue
        
    return pd.Series(ndvi_val, copy=False)

In [ ]:
image_collection = 'LANDSAT/LC08/C01/T1_TOA'
start_date = '2015-01-01'
end_date = '2015-12-31'

In [ ]:
df_ndvi = get_ndvi_index(df_generated_points, image_collection, start_date, end_date)

После определения порога, его можно использовать для фильтрации не подходящих точек. Далее выбирается интересующая область, задается количество точек. Для каждой точки вычисляется значение NDVI индекса средствами API, предоставляемыми GEE. После чего точки фильтруются по определенному пороговому значению NDVI, результат фильтрации проиллюстрирован

![This is a alt text.](image/ndvi_filter.png "This is a sample image.")